## Making the text processor
**STEPS**:
- First the class Data Structure is returned 
- Each class may have multiple instances in the boxes_dict 
- Let us say that we have a class in the dictionary with the Confidence value for each box and the (xo,yo,w,h) of the box 
- For each class there will be a **list**
- Each list element -
    - Contains the confidence of the Box 
    - Box drawing information

In [9]:
import numpy as np 

In [41]:
file_ind = open("../Data/REQUIRED_NAMES.txt",'r')
all_file = open("files/coco.names")
# read 
indoor_names = file_ind.read().splitlines()
all_names = all_file.read().splitlines()
ind_name, all_name = {},{}

for j,k in enumerate(all_names):
    all_name[j] = k
    if(k in indoor_names):
        ind_name[j] = k
# print(ind_name,all_name)

# make an empty list for each label 
store_info = {}
for k in ind_name.values():
    store_info[k] = []

# process dictionary 
def process_box_dict(boxes_dict):
    for c_index,boxes in boxes_dict.items():
        if c_index not in ind_name:
            continue
        if(len(boxes) == 0):
            continue 
        else:
            class_name = ind_name[c_index]
            # now return the 2 highest confidence object and the nearest
            distance = process_class(boxes)
            
            # this distance var contains the distance to the class 
            # in inches 
            
            distance = distance/12 # it is a numpy array 
            for k in distance:
                text = "There is a "+class_name+" "+str(distance)+" ahead of you"
                store_info[class_name].append((distance,text))
                
    # parse and keep the 2 most nearest labels 
    # to-do
    

def get_orientation(x1,x2,center):
    '''Returns orientation of the object with some 
       threshold'''
    
    object_mid = (x1+x2)//2 
    
    thres = center//8
    
    left = center - thres 
    right = center + thres 
    
    if(object_mid < left):
        return "left" 
    elif(object_mid > right):
        return "right" 
    else:
        return "ahead"
    
    
#process the class
def process_class(boxes):
    # boxes must contain elements like (xo,yo,w,h)
    c = 0
    for box in boxes:
        xy,confidence = box[:4],box[4]
        # to do...
        
        '''xy[0],xy[1] : (x,y) : top left coordinate 
           xy[2],xy[3] : (x,y) : bottom right coordinate'''
        top_left = (xy[0],xy[1])
        bottom_right = (xy[2],xy[3])
        
        '''Find the width and height of the bounding box'''
        width = abs(top_left[0] - bottom_right[0])
        height = abs(top_left[1] - bottom_right[1])
        
        # Find if the object is towards left, right or the center 
        x1,x2 = top_left[0],bottom_right[0]
        orient = get_orientation(x1,x2,416//2)
        
        # feed these into the distance formula given 
        D = ((2*3.14*180)/(width + height*360))* 1000 + 3

        print("Distance from camera is ",D/12,"feet")
        print("Object is on the ",orient)
        
    

In [44]:
b = [[306.77042  , 135.80151  , 353.52597  , 357.2086   ,   0.866853 ]]

In [45]:
r = process_class(b)

Distance from camera is  1.4311422299756782 feet
Object is on the  right
